In [0]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import random #데이터 전처리
from pandas import DataFrame #데이터 전처리
from collections import Counter #데이터 전처리

from tqdm import tqdm #시간 측정용

from sklearn.feature_extraction.text import CountVectorizer # model setting
from sklearn.model_selection import train_test_split  # model setting

from sklearn.naive_bayes import MultinomialNB  # model 관련
from sklearn.metrics import roc_auc_score  # model 성능 확인

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
cd /content/gdrive/My Drive/14th data

/content/gdrive/My Drive/14th data


In [0]:
train = pd.read_csv("train.csv") #해당 14th data의 csv 파일 중 train.csv 불러오기
train.head(2)

,id,year_month,text,smishing
0,0,2017-01-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0


In [0]:
Counter(train['smishing'])

Counter({0: 277242, 1: 18703})

In [0]:
test = pd.read_csv("public_test.csv")
test.head(2)

,id,year_month,text
0,340000,2019-01-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...


In [0]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.


In [0]:
cd Mecab-ko-for-Google-Colab/

/content/gdrive/My Drive/14th data/Mecab-ko-for-Google-Colab


In [0]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 53.4MB/s 
     |████████████████████████████████| 2.3MB 43.7MB/s 
     |████████████████████████████████| 92kB 12.0MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-01-12 04:40:46--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.1, 18.205.93.0, 18.205.93.2, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz

In [0]:
random.seed(2019) #반복 수행시에도 동일한 결과 나올 수 있도록 시드 번호 지정
train_nsm_list=list(train[train['smishing']!=1].index)
train_nsmishing=random.sample(train_nsm_list, 15000 )

random.seed(2019)
train_sm_list=list(train[train['smishing']==1].index)
train_smishing=random.sample(train_sm_list, 1000 ) #0.066과 제일 비슷하게 나올 수 있도록  train data under sampling

In [0]:
train_xx=train.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True) 

train_yy=DataFrame(train['smishing'],columns=['smishing']) 
train_yyy=train_yy.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True)

In [0]:
test['smishing']=2 #train data와 동일한 형태 생성을 위해 임의의 숫자를 추가 #이후 스미싱 여부 확률 값으로 덮어 씌워짐
test_xx=DataFrame(test['text'])
test_yyy=DataFrame(test['smishing'])

In [0]:
train_xx.shape,train_yyy.shape,test_xx.shape,test_yyy.shape

((16000, 4), (16000, 1), (1626, 1), (1626, 1))

In [0]:
from konlpy.tag import Okt  
okt=Okt()

In [0]:
train_doc = [ ( okt.pos(x), y ) for x, y in tqdm( zip( train_xx['text'], train_yyy['smishing'] ) ) ]

16000it [03:31, 75.68it/s]


In [0]:
test_doc = [ ( okt.pos(x), y ) for x, y in tqdm( zip( test_xx['text'], test_yyy['smishing'] ) )  ]

1626it [00:29, 54.33it/s]


In [0]:
stopwords = ['께서','께','하여','해','거나','면','까지','도','하고','에게','혹은','and','or','따라','등','으로서','에는','과','으로','이라고','와','에','로','에서','1','2','3','4','5','6','의','나','수','!!','...','()',')(','.)','에도','..','!-','null','zx','!!(','))','!!!','?()','ㄹ ㅣ','.?',':?','!.',').(',')------------','.---------------','.------------','------------','----------------------------------------------','쓰시','-------------------------','!!!-','):','.-------------',')---------------------------------','----------------------------------------','ㅡㅡㅡㅡㅡㅡㅡㅡ','!)','.--------------','..(','.()','ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ','(:','!!(', '듀최',')-------------','-----',')-?','))(',')??','.!!','(...','XX','XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는'] #필요없는 단어 리스트

def get_couple(_words): #필요없는 단어들 없애는 함수
    global stopwords
    _words = [x for x in _words if x[0] not in stopwords]
    l = len(_words)
    for i in range(l-1):
        yield _words[i][0], _words[i+1][0]

In [0]:
X_train, Y_train = [], []
for lwords in train_doc:
    Y_train.append(lwords[1])
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_train.append(" ".join(temp))

In [0]:
X_test = []
for lwords in test_doc:
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_test.append(" ".join(temp))

In [0]:
X_train[0:1]

['광고.S S.B B.I I.안녕하세요 안녕하세요.하루하루 하루하루.힘든 힘든.업무 업무.스트레스 스트레스.고생 고생.많으십니다 많으십니다.여유 여유.시간 시간.내어 내어.읽어 읽어.주신다면 주신다면.도움 도움.되실거라 되실거라.생각 생각.되어 되어.보내 보내.드립니다 드립니다.2017년 2017년.하반기 하반기.정부 정부.지원 지원.상품 상품.개편 개편.되어 되어.안 안.내 내.드립니다 드립니다.금년 금년.낮은 낮은.금리 금리.대 대.출금 출금.이용 이용.하실수 하실수.있는 있는.마지막 마지막.기회 기회.입니다 입니다.고민 고민.하지 하지.마시고 마시고.전화 전화.주세요 주세요.본 본.상품 상품.서 서.민지원 민지원.대출 대출.정부 정부.지원 지원.S S.B B.I I.판매 판매.하는 하는.대출 대출.상품 상품.입니다 입니다.상품 상품.특징 특징.무 무.담보 담보.무 무.보증 보증.무 무.수수료 수수료.신용등급 신용등급.상관없이 상관없이.최고 최고.한도 한도.90004 90004.최하 최하.금리 금리.4.24% 4.24%.9.89% 9.89%.상환 상환.수수료 수수료.없이 없이.자유롭게 자유롭게.상환 상환.가능 가능.기존 기존.대출 대출.저 저.금리 금리.전환 전환.가능 가능.자격 자격.조건 조건.만 만.21 21.세 세.65 65.세 세.직장인 직장인.사업자 사업자.프리랜서 프리랜서.주부 주부.최근 최근.부결 부결.자 자.다중 다중.채무자 채무자.가능 가능.신청 신청.기간 기간.9월 9월.29일 29일.마감 마감.접수 접수.자 자.많으니 많으니.빠른 빠른.신청 신청.바랍니다 바랍니다.신청 신청.방법 방법.상담 상담.가능하실 가능하실.때 때.카톡 카톡.추가 추가.또는 또는.전화 전화.주시 주시.상담 상담.가능하십니다 가능하십니다.상담 상담.전화 전화.카카오 카카오.톡 톡.문의 문의.아이디 아이디.bb bb.745 745.입니다 입니다.해당 해당.되지 되지.않으시거나 않으시거나.거부 거부.원하시는 원하시는.분 분.들 들.거부 거부.라고 라고.회신 회신.주신다

In [0]:
import numpy as np
X_train = np.asarray(X_train) 
y_train = np.asarray(Y_train) 
X_test = np.asarray(X_test) 

In [0]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K

sess = tf.Session()
K.set_session(sess)
# 세션 초기화. 이는 텐서플로우 개념.

elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
# 텐서플로우 허브로부터 ELMo를 다운로드

sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

Using TensorFlow backend.


In [0]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict=True, signature="default")["default"]

In [0]:
from keras.models import Model
from keras.layers import Dense, Lambda, Input

input_text = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(128, activation='relu')(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(hidden_layer)
model = Model(inputs=[input_text], outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
history = model.fit(X_train, Y_train, epochs=3, batch_size=60)

Epoch 1/3



 1140/16000 [=>............................] - ETA: 5:36:01 - loss: 0.2226 - acc: 0.8860

In [0]:
y_test_pred1=model.predict(X_test)
y_test_pred1_one= [ i[0]  for i in y_test_pred1]

In [0]:
y_test_pred1_one[2]

0.003959596

In [0]:
cd /content/gdrive/My Drive/14th data

/content/gdrive/My Drive/14th data


In [0]:
submission = pd.DataFrame()
submission['id'] = test['id']
submission['smishing'] = y_test_pred1_one

submission.to_csv("submission.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장